# Intro

Completness check of the new 2025 data.

Excel manual check:
* Wszystkie kolumny wydaja sie ok, jedynie 'Czas pracy (co godzine)[%]' jest pusta we wszystkich plikach

# 0. Imports

In [1]:
import pandas as pd

import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
import os

from scipy.interpolate import interp1d

from moepy import lowess
from scipy.optimize import curve_fit # for power curve fitting
import sys
# appending a path with my modules
sys.path.append(r'W:\PD-Engines\Engine Integration Validation\Department\Data Analytics\Wiktor\Python')
import Helpers

sns.set_theme()

color_dict = {'WTG007_old' : sns.color_palette()[0], #bluish
              'WTG007_new' : sns.color_palette()[3], #redish
              'WTG008' : sns.color_palette()[2],    #green
              'WTG007_predicted_w_old_LEP' : sns.color_palette()[4]}

# 0.1 Proto

In [2]:
file_path = Helpers.get_files(r'data\2025', extension='xlsx')[0]
df =  pd.read_excel(file_path)

In [3]:
df.describe()

,Sygnatura czasowa,Temp otoczenia (avg) [°C],Kierunek wiatru Główny wiatrowskaz (avg) [°],Prędkość wiatru Główny anemometr (avg) [m/s],Kierunek Gondola (avg) [°],Całkowita produkcja energii czynnej (sum) [Wh],Moc czynna (avg) [W],Bezwzględny kierunek wiatru LaPM (avg) [°],Względny kierunek wiatru Główny wiatrowskaz (avg) [°],Czas pracy (co godzinę) [%]
count,52555,51916.000000,51916.000000,51916.000000,51916.000000,51916.000000,5.191600e+04,51916.0,51916.000000,0.0
mean,2023-07-02 12:26:10.731614464,10.763483,195.461688,7.190529,196.845250,136734.468468,8.204100e+05,0.0,-0.334329,NaN
min,2023-01-01 00:10:00,-3.000000,0.000000,0.300000,0.000000,-5891.000000,-3.529900e+04,0.0,-180.000000,NaN
25%,2023-04-02 06:55:00,5.000000,126.000000,4.300000,128.000000,19389.500000,1.162748e+05,0.0,-2.000000,NaN
50%,2023-07-02 12:40:00,10.000000,215.000000,6.700000,216.000000,84343.000000,5.057990e+05,0.0,0.000000,NaN
75%,2023-10-01 18:25:00,17.000000,266.000000,9.200000,268.000000,213683.000000,1.282000e+06,0.0,2.000000,NaN
max,2024-01-01 01:00:00,32.000000,360.000000,25.400000,360.000000,500531.000000,3.003100e+06,0.0,180.000000,NaN
std,NaN,7.051447,90.222617,3.680086,90.702429,144358.725693,8.661559e+05,0.0,18.257322,NaN


In [4]:
%qtconsole

# 1.0 Load all files

In [5]:
df_list = []
for file_path in Helpers.get_files(r'data\2025', extension='xlsx'):
    temp_df = pd.read_excel(file_path)
    temp_df['WTG'] = os.path.basename(file_path).split('-')[0].strip()
    df_list.append(temp_df)

In [6]:
df

,Sygnatura czasowa,Temp otoczenia (avg) [°C],Kierunek wiatru Główny wiatrowskaz (avg) [°],Prędkość wiatru Główny anemometr (avg) [m/s],Kierunek Gondola (avg) [°],Całkowita produkcja energii czynnej (sum) [Wh],Moc czynna (avg) [W],Bezwzględny kierunek wiatru LaPM (avg) [°],Względny kierunek wiatru Główny wiatrowskaz (avg) [°],Czas pracy (co godzinę) [%]
0,2023-01-01 00:10:00,14.0,220.0,13.3,221.0,444355.0,2709000.0,0.0,-1.0,NaN
1,2023-01-01 00:20:00,14.0,222.0,13.1,221.0,395165.0,2344300.0,0.0,1.0,NaN
2,2023-01-01 00:30:00,14.0,225.0,12.8,224.0,299918.0,1799400.0,0.0,1.0,NaN
3,2023-01-01 00:40:00,14.0,223.0,12.9,227.0,300029.0,1800199.0,0.0,-4.0,NaN
4,2023-01-01 00:50:00,15.0,224.0,13.8,223.0,300043.0,1800199.0,0.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
52550,2023-12-31 23:30:00,4.0,160.0,7.6,157.0,142877.0,858599.0,0.0,3.0,NaN
52551,2023-12-31 23:40:00,4.0,163.0,7.8,163.0,143032.0,856299.0,0.0,0.0,NaN
52552,2023-12-31 23:50:00,4.0,161.0,7.4,164.0,119747.0,717099.0,0.0,-3.0,NaN
52553,2024-01-01 00:00:00,5.0,160.0,8.2,164.0,145256.0,874599.0,0.0,-4.0,NaN


# 2.0 Basic checks

In [7]:
print("Percent of missing data:")
df.isna().sum() / len(df) * 100

Percent of missing data:


Sygnatura czasowa                                          0.000000
Temp otoczenia (avg) [°C]                                  1.215869
Kierunek wiatru Główny wiatrowskaz (avg) [°]               1.215869
Prędkość wiatru Główny anemometr (avg) [m/s]               1.215869
Kierunek Gondola (avg) [°]                                 1.215869
Całkowita produkcja energii czynnej (sum) [Wh]             1.215869
Moc czynna (avg) [W]                                       1.215869
Bezwzględny kierunek wiatru LaPM (avg) [°]                 1.215869
Względny kierunek wiatru Główny wiatrowskaz (avg) [°]      1.215869
Czas pracy (co godzinę) [%]                              100.000000
dtype: float64

In [8]:
df = pd.concat(df_list)
df.describe()

,Sygnatura czasowa,Temp otoczenia (avg) [°C],Kierunek wiatru Główny wiatrowskaz (avg) [°],Prędkość wiatru Główny anemometr (avg) [m/s],Kierunek Gondola (avg) [°],Całkowita produkcja energii czynnej (sum) [Wh],Moc czynna (avg) [W],Bezwzględny kierunek wiatru LaPM (avg) [°],Względny kierunek wiatru Główny wiatrowskaz (avg) [°],Czas pracy (co godzinę) [%]
count,420723,410353.000000,410353.00000,410353.000000,410353.000000,410353.000000,4.103530e+05,410353.0,410353.000000,0.0
mean,2024-05-02 00:17:34.925449984,11.366258,194.41717,6.851840,196.543764,127641.605184,7.658280e+05,0.0,-0.243298,NaN
min,2023-01-01 00:10:00,-6.000000,0.00000,0.000000,0.000000,-6541.000000,-3.920000e+04,0.0,-180.000000,NaN
25%,2023-09-01 12:30:00,5.000000,125.00000,4.200000,128.000000,17976.000000,1.078000e+05,0.0,-2.000000,NaN
50%,2024-05-02 00:40:00,11.000000,210.00000,6.500000,212.000000,78718.000000,4.722000e+05,0.0,0.000000,NaN
75%,2024-12-31 12:00:00,17.000000,266.00000,8.900000,269.000000,197521.000000,1.185000e+06,0.0,2.000000,NaN
max,2025-09-01 01:00:00,34.000000,360.00000,25.900000,360.000000,506560.000000,3.003699e+06,0.0,180.000000,NaN
std,NaN,6.927014,90.54942,3.392732,90.985082,136898.276143,8.213706e+05,0.0,21.166466,NaN


Wartosci sa w oczekiwanych przedziałach, oprócz:
* 'Czas pracy (co godzinę) [%]' jest pusta we wszystkich plikach
* 'Względny kierunek wiatru Główny wiatrowskaz (avg) [°]' ma zawsze wartosc 0?

In [9]:
#Check per WTG
describe_df = []
for wtg in df.WTG.unique():
    describe_df.append(df[df.WTG==wtg].describe().assign(WTG=wtg))
describe_df = pd.concat(describe_df)
describe_df

,Sygnatura czasowa,Temp otoczenia (avg) [°C],Kierunek wiatru Główny wiatrowskaz (avg) [°],Prędkość wiatru Główny anemometr (avg) [m/s],Kierunek Gondola (avg) [°],Całkowita produkcja energii czynnej (sum) [Wh],Moc czynna (avg) [W],Bezwzględny kierunek wiatru LaPM (avg) [°],Względny kierunek wiatru Główny wiatrowskaz (avg) [°],Czas pracy (co godzinę) [%],WTG
count,140241,139090.000000,139090.000000,139090.000000,139090.000000,139090.000000,1.390900e+05,139090.0,139090.000000,0.0,WTG12
mean,2024-05-02 00:17:34.925449728,11.147602,196.302897,6.927461,198.345819,127980.479251,7.678648e+05,0.0,-0.184593,NaN,WTG12
min,2023-01-01 00:10:00,-4.000000,0.000000,0.000000,0.000000,-5891.000000,-3.629900e+04,0.0,-180.000000,NaN,WTG12
25%,2023-09-01 12:30:00,5.000000,126.000000,4.200000,130.000000,16895.000000,1.014000e+05,0.0,-2.000000,NaN,WTG12
50%,2024-05-02 00:40:00,11.000000,212.000000,6.500000,214.000000,77117.000000,4.627990e+05,0.0,0.000000,NaN,WTG12
75%,2024-12-31 12:00:00,17.000000,269.000000,8.900000,272.000000,198104.750000,1.188099e+06,0.0,2.000000,NaN,WTG12
max,2025-09-01 01:00:00,33.000000,360.000000,25.400000,360.000000,500531.000000,3.003100e+06,0.0,180.000000,NaN,WTG12
std,NaN,7.069291,90.871298,3.488877,91.222666,138617.329942,8.316940e+05,0.0,20.548249,NaN,WTG12
count,140241,132332.000000,132332.000000,132332.000000,132332.000000,132332.000000,1.323320e+05,132332.0,132332.000000,0.0,WTG15
mean,2024-05-02 00:17:34.925449728,11.932700,193.090454,6.675791,195.132190,123562.480798,7.413643e+05,0.0,-0.063734,NaN,WTG15


Analiza kosztowała mnie 1hr 20min. 